<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import the function from the script
import myStocks_Portfolio_Analysis_v2

# Call the function to get the DataFrame
df_portfolio_features = myStocks_Portfolio_Analysis_v2.get_portfolio_features_v2()
cols = ['Symbol', 'AvgCost', 'Close', 'FTT', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_portfolio_features = df_portfolio_features[cols]

Run date time (IST): 2024-09-22 14:56:27
-------------------
qualified stocks: 86
with latest results: 82
still star stocks: 41
-------------------
Initial Investment:  93.12 L
CY Investment:  1.06 C
Reserve:  1.50 K
Current:  1.23 C
-------------------
Today PnL: 1.22 L (1.01%)
Current PnL: 9.85 L (9.27%)
CY Booked + Current PnL: 16.18 L (15.23%)
-------------------
Total profit:  13.80 L
Total loss:  -3.94 L
-------------------
Total Booked + Current PnL: 29.29 L (31.46%)
Total Booked PnL: 19.44 L (20.88%)
Curr Year Booked PnL: 6.33 L (5.15%)
Prev Year Booked PnL: 13.11 L (14.08%)
Est FTT:  1.77 C
Est FTT PnL: 54.09 L (44.04%)
-------------------
Est LTT:  2.25 C
Est LTT PnL: 1.03 C (83.55%)
Deployed:  93.12 L
Current:  1.23 C
CAGR/XIRR %: 30.28%


In [4]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,AvgCost,Close,FTT,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROSSARI,753.22,912.90,965.00,36.05,140587.0,24591.0,8028.0,-0.21,21.20,5.71,28.12,24.0,3.06,1.14,36.20,XSP,NTT,CHEMICALS
71,SONACOMS,578.05,748.35,800.00,74.42,130961.0,29802.0,9036.0,2.72,29.46,6.90,38.40,41.0,3.30,1.07,47.74,AR,NTT,AUTO
46,JUBLFOOD,487.35,703.40,750.00,89.53,157562.0,48396.0,10431.0,1.14,44.33,6.62,53.89,34.0,4.64,1.28,63.54,AR,NTT,RESTAURANTS
56,PIDILITIND,2482.94,3287.10,3483.00,77.91,190652.0,46641.0,11363.0,1.33,32.39,5.96,40.28,27.0,4.10,1.55,43.93,X40,BTT,CHEMICALS
23,DABUR,523.35,667.55,735.00,75.58,116821.0,25235.0,11799.0,0.43,27.55,10.10,40.44,31.0,2.14,0.95,35.38,X40,BTT,FMCG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,CAMPUS,294.86,357.45,631.85,98.84,203389.0,35614.0,156142.0,2.03,21.23,76.77,114.29,187.0,0.23,1.66,66.64,XSP,ATH,FOOTWEAR
82,VAIBHAVGBL,350.21,306.35,670.00,2.33,132956.0,-19035.0,157819.0,-0.63,-12.52,118.70,91.31,-19.0,-0.12,1.08,11.33,XR,NTT,JEWELLERY
11,BANDHANBNK,223.20,210.04,400.00,73.26,205209.0,-12857.0,185591.0,-0.62,-5.90,90.44,79.21,-7.0,-0.07,1.67,22.13,XSP,NTT,BANKS
75,TANLA,991.56,939.05,2007.31,44.19,207530.0,-11605.0,236086.0,1.25,-5.30,113.76,102.44,-7.0,-0.05,1.69,17.43,AR,ATH,IT


In [5]:
# AR analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AR') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,AvgCost,Close,FTT,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SONACOMS,578.05,748.35,800.00,74.42,130961.0,29802.0,9036.0,2.72,29.46,6.90,38.40,41.0,3.30,1.07,47.74,AR,NTT,AUTO
46,JUBLFOOD,487.35,703.40,750.00,89.53,157562.0,48396.0,10431.0,1.14,44.33,6.62,53.89,34.0,4.64,1.28,63.54,AR,NTT,RESTAURANTS
50,LAOPALA,379.50,381.25,582.00,97.67,40031.0,184.0,21080.0,0.45,0.46,52.66,53.36,NaN,0.01,0.33,28.00,AR,BTT,CERAMICS
24,DEN,52.79,54.64,75.00,66.28,87424.0,2960.0,32574.0,3.58,3.50,37.26,42.07,4.0,0.09,0.71,23.20,AR,NTT,ENTERTAINMENT
30,HAPPSTMNDS,919.80,800.00,1496.99,45.35,84000.0,-12579.0,73181.0,0.98,-13.02,87.12,62.75,-18.0,-0.17,0.68,7.86,AR,ATH,IT
75,TANLA,991.56,939.05,2007.31,44.19,207530.0,-11605.0,236086.0,1.25,-5.30,113.76,102.44,-7.0,-0.05,1.69,17.43,AR,ATH,IT


In [6]:
# for average up
tmp_df = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 15) & (df_portfolio_features['FTT%'] > 15) ].sort_values(by = 'RSP', ascending=False)
data_table.DataTable(tmp_df, include_index=False)

,Symbol,AvgCost,Close,FTT,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JMFINANCIL,82.95,126.23,172.00,100.00,148573.0,50941.0,53873.0,3.22,52.18,36.26,107.35,96.0,0.95,1.21,80.05,XR,NTT,FINANCE
19,CAMPUS,294.86,357.45,631.85,98.84,203389.0,35614.0,156142.0,2.03,21.23,76.77,114.29,187.0,0.23,1.66,66.64,XSP,ATH,FOOTWEAR
35,HINDUNILVR,2443.56,2977.60,3513.00,80.23,187589.0,33645.0,33729.0,2.26,21.86,17.98,43.77,34.0,1.00,1.53,37.03,X40,BTT,FMCG
73,SUNTV,536.47,807.20,1092.00,48.84,147718.0,49544.0,52115.0,-0.28,50.47,35.28,103.55,45.0,0.95,1.20,42.37,X40N,BTT,ENTERTAINMENT
40,INFY,1436.24,1905.75,2275.00,46.51,198198.0,48829.0,38411.0,0.61,32.69,19.38,58.40,47.0,1.27,1.61,41.53,X40,BTT,IT
15,BAYERCROP,4346.57,6246.10,8325.00,39.53,99938.0,30393.0,33259.0,-2.39,43.70,33.28,91.53,33.0,0.91,0.81,31.80,X40N,BTT,CHEMICALS
34,HDFCLIFE,560.54,711.10,897.00,38.37,187019.0,39597.0,48887.0,1.37,26.86,26.14,60.02,100.0,0.81,1.52,33.50,X40,BTT,INSURANCE
77,TEAMLEASE,2557.47,3131.05,4990.00,34.88,131504.0,24090.0,78074.0,1.09,22.43,59.37,95.11,20.0,0.31,1.07,34.63,X40N,NTT,MISC
27,GICRE,335.06,393.65,583.00,32.56,172025.0,25604.0,82744.0,2.51,17.49,48.10,74.00,148.0,0.31,1.40,88.53,XSP,BTT,INSURANCE
52,LUXIND,1448.19,2180.30,2880.00,25.58,207129.0,69551.0,66468.0,0.95,50.55,32.09,98.87,41.0,1.05,1.69,103.13,X40N,NTT,TEXTILES


In [7]:
# XSP analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'XSP') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,AvgCost,Close,FTT,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROSSARI,753.22,912.90,965.00,36.05,140587.0,24591.0,8028.0,-0.21,21.20,5.71,28.12,24.0,3.06,1.14,36.20,XSP,NTT,CHEMICALS
25,EASEMYTRIP,41.84,41.64,52.80,70.93,51842.0,-249.0,13894.0,2.92,-0.48,26.80,26.20,NaN,-0.02,0.42,9.15,XSP,NTT,TRAVEL
7,ATULAUTO,665.70,648.25,844.00,37.21,53805.0,-1448.0,16249.0,-0.56,-2.62,30.20,26.78,NaN,-0.09,0.44,36.27,XSP,NTT,AUTO
43,JIOFIN,320.24,353.90,387.00,68.60,174827.0,16628.0,16346.0,2.09,10.51,9.35,20.85,139.0,1.02,1.42,71.30,XSP,BTT,FINANCE
81,UNIONBANK,123.96,123.43,163.00,23.26,54803.0,-235.0,17570.0,0.66,-0.43,32.06,31.49,NaN,-0.01,0.45,33.71,XSP,NTT,BANKS
16,BERGEPAINT,558.54,622.85,680.00,83.72,197443.0,20386.0,18125.0,0.38,11.51,9.18,21.75,10.0,1.12,1.61,37.56,XSP,NTT,PAINTS
57,QUESS,716.17,864.35,986.00,96.51,151261.0,25931.0,21282.0,2.57,20.69,14.07,37.68,NaN,1.22,1.23,113.37,XSP,NTT,MISC
17,BOROLTD,348.25,407.10,462.00,22.09,163654.0,23658.0,22077.0,-0.33,16.90,13.49,32.66,90.0,1.07,1.33,35.56,XSP,NTT,GLASS
69,SHALBY,271.92,282.85,327.00,58.14,156416.0,6044.0,24417.0,3.12,4.02,15.61,20.26,19.0,0.25,1.27,31.60,XSP,NTT,HEALTHCARE
72,STARHEALTH,596.56,618.60,761.00,54.65,165785.0,5907.0,38164.0,2.15,3.69,23.02,27.56,4.0,0.15,1.35,23.91,XSP,NTT,INSURANCE
